# PhysioNet/Computing in Cardiology Challenge 2020
## Classification of 12-lead ECGs
### Synthetic Noise Generation

# Setup Notebook

In [13]:
# Import 3rd party libraries
import os
import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from ipywidgets import interact, fixed

# Import local Libraries
sys.path.insert(0, os.path.dirname(os.path.abspath(os.getcwd())))
from kardioml import DATA_PATH, FS, LABELS_LOOKUP, ECG_LEADS

# Configure Notebook
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Synthetic Noise Generation

In [38]:
def gen_bw_noise(sig_t, sig_v, fs, amplitude=0.1):
    """
    Adds baseline wandering to the input signal.

    Parameters:
      fs: Wandering frequency in Hz
      amplitude: Wandering amplitude
    """
    w = 2*np.pi*fs
    mod_v = sig_v + amplitude * np.sin(w*sig_t + 20)
    return [sig_t, mod_v]

def gen_am_noise(sig_t, sig_v, fs, amplitude=0.1):
    """
    Adds amplitude modulation noise to the input signal.

    Parameters:
      fs: Modulation frequency in Hz
      amplitude: Modulation amplitude
    """
    w = 2*np.pi*fs
    mod_v = detrend(sig_v)*((1/amplitude)+np.cos(w*sig_t));
    return [sig_t, mod_v]

def gen_fm_noise(sig_t, sig_v, fs, amplitude=0.05):
    """
    Adds frequency modulation noise to the input signal.

    Parameters:
      fs: Modulation frequency in Hz
      amplitude: Modulation amplitude
    """
    w = 2*np.pi*fs
    mod_t = sig_t + amplitude*np.sin(w*sig_t)
    mod_v = np.interp(mod_t, sig_t, sig_v)
    return [sig_t, mod_v]

def generate_noise_burst(waveform):
    """Generate multiple noise bursts."""
    
    

# Plot Synthetic Noise

In [50]:
def waveform_plot(filename_id, filenames, path):
    """Plot measure vs time."""
    # Get filename
    filename = filenames[filename_id]

    # Import waveforms
    waveforms = np.load(os.path.join(path, '{}.npy'.format(filename)))

    # Time array
    time = np.arange(waveforms.shape[1]) * 1 / FS

    # Setup figure
    fig = plt.figure(figsize=(15, 15), facecolor='w')
    fig.subplots_adjust(wspace=0, hspace=0.05)
    ax1 = plt.subplot2grid((1, 1), (0, 0))

    # ECG
    ax1.set_title('File Name: {}'.format(filename), fontsize=20, loc='left', x=0)
    shift = 0
    for channel_id in range(waveforms.shape[0]):
        ax1.plot(time, waveforms[channel_id, :] + shift, '-k', lw=2)
        ax1.text(0.1, 0.25 + shift, ECG_LEADS[channel_id], color='red', fontsize=16, ha='left')
        shift += 3

    # Axes labels
    ax1.set_xlabel('Time, seconds', fontsize=24)
    ax1.set_ylabel('ECG Amplitude, mV', fontsize=24)
    ax1.set_xlim([time.min(), time.max()])
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)

    plt.show()


def waveform_plot_interact():
    """Launch interactive plotting widget."""
    # Set data path
    path = os.path.join(DATA_PATH, 'formatted')

    # Get filenames
    filenames = [filename.split('.')[0] for filename in os.listdir(path) if 'npy' in filename]

    interact(waveform_plot,
             filename_id=(0, len(filenames) - 1, 1),
             filenames=fixed(filenames),
             path=fixed(path))

In [51]:
# Plot waveforms
waveform_plot_interact()

interactive(children=(IntSlider(value=3438, description='filename_id', max=6876), Output()), _dom_classes=('wi…

In [ ]:
5617